<a href="https://colab.research.google.com/github/SrutiRaniPanda/AI-Personalized-Reading-Assistant/blob/main/AI_Reading_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install nltk spacy pyttsx3

# Download spaCy English model
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 98.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import nltk
import spacy
import json
import os
import re
from collections import defaultdict
import random
from datetime import datetime
import threading
import time
from IPython.display import display, HTML, clear_output, Audio
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import base64
import io
import subprocess
import sys


In [8]:
# Install required packages
def install_requirements():
    print("Installing required packages...")

    # Install packages
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "nltk", "spacy", "gtts", "pygame", "pydub"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ipywidgets"])

    # Download spaCy model
    try:
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    except:
        print("SpaCy model download failed. Trying alternative...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1.tar.gz"])

    print("Installation complete!")

# Run installation
install_requirements()

# Download required NLTK data with error handling
def download_nltk_data():
    import nltk
    try:
        nltk.download('punkt')
        nltk.download('cmudict')
        nltk.download('stopwords')
        nltk.download('averaged_perceptron_tagger')
        # Additional punkt data needed for sentence tokenization
        nltk.download('punkt_tab')
    except Exception as e:
        print(f"Error downloading NLTK data: {e}")
        print("Trying alternative download method...")
        try:
            nltk.download('all')  # As a last resort, download all data
        except Exception as e:
            print(f"Failed to download NLTK data: {e}")
            print("Some features may not work properly.")

download_nltk_data()

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import cmudict, stopwords
from nltk.probability import FreqDist
from nltk import pos_tag

# For text-to-speech in Colab
from gtts import gTTS
import pygame
from pydub import AudioSegment
from pydub.playback import play

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("SpaCy model not found. Please run the installation cell again.")
    nlp = None

# Rest of your code remains the same...

Installing required packages...
Installation complete!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Install required packages
def install_requirements():

    print("Installing required packages...")

    # Install packages
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "nltk", "spacy", "gtts", "pygame", "pydub"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ipywidgets"])

    # Download spaCy model
    try:
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    except:
        print("SpaCy model download failed. Trying alternative...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1.tar.gz"])

    print("Installation complete!")

In [17]:



# Run installation
install_requirements()

# Download required NLTK data
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import cmudict, stopwords
from nltk.probability import FreqDist
from nltk import pos_tag

# For text-to-speech in Colab
from gtts import gTTS
import pygame
from pydub import AudioSegment
from pydub.playback import play

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("SpaCy model not found. Please run the installation cell again.")
    nlp = None

class ReadingAssistantColab:
    def __init__(self):
        """Initialize the Reading Assistant for Google Colab"""
        self.pronunciation_dict = cmudict.dict()
        self.user_data = self.load_user_data()
        self.current_article = ""
        self.difficult_words = {}
        self.simplified_text = ""
        self.current_sentences = []
        self.current_sentence_index = 0

        # Word simplification database
        self.word_simplifications = {
            'utilize': 'use', 'facilitate': 'help', 'demonstrate': 'show',
            'substantial': 'large', 'magnificent': 'great', 'extremely': 'very',
            'immediately': 'right away', 'consequently': 'so', 'furthermore': 'also',
            'nevertheless': 'but', 'approximately': 'about', 'comprehensive': 'complete',
            'significantly': 'much', 'particularly': 'especially', 'establish': 'set up',
            'maintain': 'keep', 'acquire': 'get', 'construct': 'build',
            'accomplish': 'do', 'investigate': 'study', 'numerous': 'many',
            'essential': 'important', 'adequate': 'enough', 'eliminate': 'remove',
            'extraordinary': 'amazing', 'magnificent': 'great', 'tremendous': 'huge',
            'sophisticated': 'advanced', 'fundamental': 'basic', 'theoretical': 'idea-based',
            'implementation': 'putting into action', 'collaboration': 'working together',
            'optimization': 'making better', 'revolutionary': 'game-changing'
        }

        print(" Personalized Reading Assistant - Colab Version")
        print("="*50)
        self.setup_interface()

    def load_user_data(self):
        """Load user progress data from JSON file"""
        try:
            with open('user_data.json', 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {
                'xp': 0,
                'level': 1,
                'learned_words': {},
                'reading_history': [],
                'quiz_scores': []
            }

    def save_user_data(self):
        """Save user progress data to JSON file"""
        with open('user_data.json', 'w') as f:
            json.dump(self.user_data, f, indent=2)

    def setup_interface(self):
        """Setup the interactive interface"""
        # Create text input area
        self.text_input = widgets.Textarea(
            value='',
            placeholder='Paste your article text here...',
            description='Article:',
            layout=widgets.Layout(width='100%', height='200px')
        )

        # Create buttons
        self.analyze_btn = widgets.Button(
            description=' Analyze Text',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )

        self.simplify_btn = widgets.Button(
            description=' Toggle Simplified',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )

        self.read_btn = widgets.Button(
            description='🔊 Read Aloud',
            button_style='success',
            layout=widgets.Layout(width='150px')
        )

        self.quiz_btn = widgets.Button(
            description='📝 Take Quiz',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )

        self.progress_btn = widgets.Button(
            description=' View Progress',
            button_style='',
            layout=widgets.Layout(width='150px')
        )

        # Bind button events
        self.analyze_btn.on_click(self.analyze_text)
        self.simplify_btn.on_click(self.toggle_simplified)
        self.read_btn.on_click(self.read_aloud)
        self.quiz_btn.on_click(self.start_quiz)
        self.progress_btn.on_click(self.show_progress)

        # Create output area
        self.output = widgets.Output()

        # Display interface
        button_box = widgets.HBox([
            self.analyze_btn, self.simplify_btn, self.read_btn,
            self.quiz_btn, self.progress_btn
        ])

        display(self.text_input)
        display(button_box)
        display(self.output)

        # Show initial progress
        self.show_progress(None)

    def analyze_text(self, btn):
      """Analyze text for difficult words"""
      with self.output:
        clear_output()

        if not self.text_input.value.strip():
            print(" Please enter some text to analyze!")
            return

        self.current_article = self.text_input.value.strip()
        print("🔍 Analyzing text...")

        if nlp is None:
            print(" SpaCy model not loaded. Please run installation cell.")
            return

        # Process text with spaCy
        doc = nlp(self.current_article)

        # Reset difficult words
        self.difficult_words = {}
        common_words = set(stopwords.words('english'))

        # Add more common words to exclude
        common_words.update(['knowing', 'usually', 'really', 'something', 'sometimes'])

        for token in doc:
            # Skip if not alphabetic, is a stopword, or is punctuation
            if not token.is_alpha or token.is_stop or token.is_punct:
                continue

            word_lower = token.text.lower()

            # Skip very common short words
            if len(token.text) <= 5 and word_lower in common_words:
                continue

            # Check if word is in our simplification dictionary (case insensitive)
            if word_lower in self.word_simplifications:
                simplified = self.word_simplifications[word_lower]
                # Only mark as difficult if the simplified version is actually different
                if simplified != word_lower:
                    self.difficult_words[token.text] = {
                        'simplified': simplified,
                        'pos': token.pos_,
                        'definition': f"{token.pos_} - {simplified}"
                    }
            # Or if it meets advanced difficulty criteria
            elif (len(token.text) > 7 and  # Longer words
                  token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV'] and  # Content words
                  word_lower not in common_words and  # Not common
                  not any(char.isdigit() for char in token.text)):  # Not numbers
                # Add with self as simplified version
                self.difficult_words[token.text] = {
                    'simplified': token.text,  # No simplification available
                    'pos': token.pos_,
                    'definition': f"{token.pos_} - {token.text}"
                }

        # Create simplified text - replacing only words with actual simplifications
        self.simplified_text = self.current_article
        for word, info in self.difficult_words.items():
            if info['simplified'].lower() != word.lower():  # Only replace if different
                # Handle case-sensitive replacement
                self.simplified_text = re.sub(
                    r'(?<!\w)' + re.escape(word) + r'(?!\w)',  # Whole word match
                    info['simplified'],
                    self.simplified_text,
                    flags=re.IGNORECASE
                )

        # Display results
        print(f" Analysis complete!")
        print(f"📊 Found {len(self.difficult_words)} difficult words")
        print("\n" + "="*50)

        # Show difficult words
        if self.difficult_words:
            print("🔴 Difficult words found:")
            for word, info in list(self.difficult_words.items())[:15]:  # Show first 15
                if info['simplified'].lower() != word.lower():
                    print(f"  • {word} → {info['simplified']} ({info['pos']})")
                else:
                    print(f"  • {word} (no simplification available)")
            if len(self.difficult_words) > 15:
                print(f"  ... and {len(self.difficult_words) - 15} more")
        else:
            print("✅ No difficult words found according to current criteria")

        # Display original text with highlights
        print("\n📄 Original Text:")
        print("-" * 30)
        self.display_text_with_highlights()

    def display_text_with_highlights(self):
        """Display text with difficult words highlighted"""
        text = self.current_article

        # Create HTML with highlighted difficult words
        html_text = text
        for word in self.difficult_words:
            pattern = r'\b' + re.escape(word) + r'\b'
            replacement = f'<span style="background-color: yellow; color: red; font-weight: bold;">{word}</span>'
            html_text = re.sub(pattern, replacement, html_text, flags=re.IGNORECASE)

        # Display as HTML
        display(HTML(f'<div style="padding: 10px; border: 1px solid #ccc; background-color: #f9f9f9;">{html_text}</div>'))

    def toggle_simplified(self, btn):
        """Toggle between original and simplified text"""
        with self.output:
            clear_output()

            if not self.current_article:
                print("❌ Please analyze text first!")
                return

            if hasattr(self, 'show_simplified') and self.show_simplified:
                print("📄 Original Text:")
                print("-" * 30)
                self.display_text_with_highlights()
                self.show_simplified = False
            else:
                print("📄 Simplified Text:")
                print("-" * 30)
                display(HTML(f'<div style="padding: 10px; border: 1px solid #ccc; background-color: #e8f5e8;">{self.simplified_text}</div>'))
                self.show_simplified = True

    def get_pronunciation(self, word):
        """Get pronunciation of a word"""
        word_lower = word.lower()
        if word_lower in self.pronunciation_dict:
            return ' '.join(self.pronunciation_dict[word_lower][0])
        return "Pronunciation not available"

    def text_to_speech(self, text, filename="temp_audio.mp3"):
        """Convert text to speech and return audio data"""
        try:
            tts = gTTS(text=text, lang='en', slow=False)
            tts.save(filename)
            return filename
        except Exception as e:
            print(f"Error in text-to-speech: {e}")
            return None

    def read_aloud(self, btn):
        """Read text aloud sentence by sentence"""
        with self.output:
            clear_output()

            if not self.current_article:
                print("❌ Please analyze text first!")
                return

            print("🔊 Reading text aloud...")
            print("Note: Audio will be generated for each sentence.")
            print("-" * 50)

            # Split into sentences
            sentences = sent_tokenize(self.current_article)

            for i, sentence in enumerate(sentences[:5]):  # Limit to first 5 sentences for demo
                print(f"\n📢 Sentence {i+1}:")
                print(f"'{sentence}'")

                # Generate audio
                audio_file = self.text_to_speech(sentence, f"sentence_{i}.mp3")
                if audio_file:
                    print(f"🎵 Audio generated: {audio_file}")
                    # In Colab, you can play audio using:
                    # display(Audio(audio_file))

                # Add learned words from this sentence
                self.process_sentence_for_learning(sentence)

                time.sleep(0.5)  # Brief pause between sentences

    def process_sentence_for_learning(self, sentence):
        """Process sentence and add difficult words to learned words"""
        words = word_tokenize(sentence.lower())
        for word in words:
            if word in [w.lower() for w in self.difficult_words.keys()]:
                self.add_learned_word(word, self.difficult_words.get(word, {}).get('simplified', word))

    def add_learned_word(self, word, simplified):
        """Add word to learned words list"""
        if word not in self.user_data['learned_words']:
            self.user_data['learned_words'][word] = {
                'simplified': simplified,
                'date': datetime.now().strftime("%Y-%m-%d"),
                'count': 1
            }
            self.user_data['xp'] += 10
            print(f"✨ New word learned: {word} → {simplified} (+10 XP)")
        else:
            self.user_data['learned_words'][word]['count'] += 1
            self.user_data['xp'] += 2
            print(f"🔄 Word reviewed: {word} (+2 XP)")

        # Update level
        self.user_data['level'] = self.user_data['xp'] // 100 + 1
        self.save_user_data()

    def generate_questions(self):
        """Generate comprehension questions based on text"""
        if not nlp:
            return self.generate_simple_questions()

        doc = nlp(self.current_article)
        questions = []

        # Extract named entities
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        # Generate entity-based questions
        for entity, label in entities[:2]:
            if label in ['PERSON', 'ORG', 'GPE']:
                questions.append({
                    'question': f"What is mentioned about {entity} in the text?",
                    'type': 'open',
                    'entity': entity
                })

        # Add general questions
        questions.extend([
            {
                'question': "What is the main topic of this text?",
                'type': 'open',
                'topic': 'main_topic'
            },
            {
                'question': "List three key points from the text.",
                'type': 'open',
                'topic': 'key_points'
            },
            {
                'question': "What did you learn from reading this text?",
                'type': 'open',
                'topic': 'learning'
            }
        ])

        return questions[:4]

    def generate_simple_questions(self):
        """Generate simple questions when spaCy is not available"""
        return [
            {
                'question': "What is the main topic of this text?",
                'type': 'open',
                'topic': 'main_topic'
            },
            {
                'question': "Summarize the text in 2-3 sentences.",
                'type': 'open',
                'topic': 'summary'
            },
            {
                'question': "What are the key points mentioned?",
                'type': 'open',
                'topic': 'key_points'
            },
            {
                'question': "What did you learn from this text?",
                'type': 'open',
                'topic': 'learning'
            }
        ]

    def start_quiz(self, btn):
        """Start comprehension quiz"""
        with self.output:
            clear_output()

            if not self.current_article:
                print("❌ Please analyze text first!")
                return

            print("📝 Starting Comprehension Quiz")
            print("=" * 40)

            questions = self.generate_questions()
            quiz_score = 0

            for i, question in enumerate(questions, 1):
                print(f"\n❓ Question {i}: {question['question']}")
                print("💭 Think about your answer...")

                # Create text input for answer
                answer_widget = widgets.Textarea(
                    value='',
                    placeholder='Type your answer here...',
                    description=f'Answer {i}:',
                    layout=widgets.Layout(width='100%', height='100px')
                )

                display(answer_widget)

                # Simple scoring (award points for non-empty answers)
                if True:  # In a real scenario, you'd validate the answer
                    quiz_score += 1
                    print(f"✅ Answer recorded! (+5 XP)")
                    self.user_data['xp'] += 5

            # Show quiz results
            print(f"\n🎉 Quiz Complete!")
            print(f"📊 Score: {quiz_score}/{len(questions)}")

            total_xp = quiz_score * 5
            print(f"✨ Total XP earned: {total_xp}")

            # Update user data
            self.user_data['quiz_scores'].append({
                'date': datetime.now().strftime("%Y-%m-%d %H:%M"),
                'score': quiz_score,
                'total': len(questions),
                'xp_earned': total_xp
            })

            self.user_data['level'] = self.user_data['xp'] // 100 + 1
            self.save_user_data()

    def show_progress(self, btn):
        """Display user progress"""
        with self.output:
            if btn is not None:  # Only clear if called from button
                clear_output()

            print("📈 Your Progress")
            print("=" * 30)
            print(f"🏆 Level: {self.user_data['level']}")
            print(f"⭐ XP: {self.user_data['xp']}")
            print(f"📚 Words Learned: {len(self.user_data['learned_words'])}")
            print(f"📝 Quizzes Taken: {len(self.user_data['quiz_scores'])}")

            # Show recent learned words
            if self.user_data['learned_words']:
                print(f"\n📖 Recent Learned Words:")
                recent_words = list(self.user_data['learned_words'].items())[-5:]
                for word, info in recent_words:
                    print(f"  • {word} → {info['simplified']} (learned: {info['date']})")

            # Show quiz history
            if self.user_data['quiz_scores']:
                print(f"\n📊 Recent Quiz Scores:")
                recent_quizzes = self.user_data['quiz_scores'][-3:]
                for quiz in recent_quizzes:
                    print(f"  • {quiz['date']}: {quiz['score']}/{quiz['total']} (+{quiz['xp_earned']} XP)")

    def get_word_info(self, word):
        """Get detailed information about a word"""
        if word in self.difficult_words:
            info = self.difficult_words[word]
            pronunciation = self.get_pronunciation(word)

            print(f"\n📝 Word Information:")
            print(f"Word: {word}")
            print(f"Simplified: {info['simplified']}")
            print(f"Type: {info['pos']}")
            print(f"Pronunciation: {pronunciation}")

            # Generate audio for pronunciation
            audio_file = self.text_to_speech(word, f"word_{word}.mp3")
            if audio_file:
                print(f"🎵 Pronunciation audio: {audio_file}")

            # Add to learned words
            self.add_learned_word(word, info['simplified'])

# Create and run the Reading Assistant
def run_reading_assistant():
    """Main function to run the Reading Assistant"""
    assistant = ReadingAssistantColab()
    return assistant

# Ask the user for their name
name = input("What is your name? ")

# Print a greeting

print(f"\n\033[1mHello, {name}!\033[0m")  # Bold text using ANSI escape codes

print("\033[1m🚀 Let's dive into the world of reading ...\033[0m")


# Provide instructions
print("\n📋 Instructions:")
print("1. Paste text in the text area above")
print("2. Click 'Analyze Text' to identify difficult words")
print("3. Use 'Toggle Simplified' to see easier version")
print("4. Click 'Read Aloud' for sentence-by-sentence reading")
print("5. Take the quiz to test comprehension")
print("6. Check your progress and earned XP!")



# Initialize the assistant
assistant = run_reading_assistant()



Installing required packages...
Installation complete!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


What is your name? SRUTI

Hello, SRUTI!
🚀 Let's dive into the world of reading ...

📋 Instructions:
1. Paste text in the text area above
2. Click 'Analyze Text' to identify difficult words
3. Use 'Toggle Simplified' to see easier version
4. Click 'Read Aloud' for sentence-by-sentence reading
5. Take the quiz to test comprehension
6. Check your progress and earned XP!
 Personalized Reading Assistant - Colab Version


Textarea(value='', description='Article:', layout=Layout(height='200px', width='100%'), placeholder='Paste you…

Output()